In [1]:
from Config import Config

config = Config()

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

data = pd.read_csv('data/vi_processed.csv')

correct_texts = []
error_texts = []

for index, row in data.iterrows():
    if len(correct_texts) == 100: break
    correct_texts.append(row.correct_text)
    error_texts.append(row.error_text)

correct_texts = correct_texts[:config.NUM_OF_INPUTS]
error_texts = error_texts[:config.NUM_OF_INPUTS]

2024-04-06 21:12:19.091833: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-06 21:12:19.101784: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 21:12:19.133730: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 21:12:19.134363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 21:12:19.888610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

word_level_tokenizer = Tokenizer(num_words=config.VOCAB_SIZE, oov_token='<UNK>', lower=True, split=' ', )

word_unk_level_tokenizer = Tokenizer(oov_token='<UNK>', lower=True, split=' ', )

character_level_tokenizer = Tokenizer(num_words=config.CHARACTER_VOCAB_SIZE, lower=True, char_level=True)

In [4]:
word_level_tokenizer.fit_on_texts(correct_texts)
word_unk_level_tokenizer.fit_on_texts(error_texts)
character_level_tokenizer.fit_on_texts(error_texts)

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_sequences = word_level_tokenizer.texts_to_sequences(error_texts)
output_sequences = word_level_tokenizer.texts_to_sequences(correct_texts)
unk_input_sequences = word_unk_level_tokenizer.texts_to_sequences(error_texts)

# Get character-level words lengths.
input_words_lengths = []

# Get character-level sequences.
character_level_input_sequences = []

for sequence in unk_input_sequences:
    character_level_input_sequence = []
    words_lengths = []
    for word_token in sequence:
        word = word_unk_level_tokenizer.index_word[word_token]
        word = character_level_tokenizer.texts_to_sequences(word)
        word_chars = [each[0] for each in word]
        character_level_input_sequence.append(word_chars)
        words_lengths.append((len(word_chars) if len(word_chars) <= config.MAX_WORD_LENGTH
                              else config.MAX_WORD_LENGTH))

    # Add padding for each word.
    character_level_input_sequence = pad_sequences(character_level_input_sequence, maxlen=config.MAX_WORD_LENGTH,
                                                   padding='post', truncating='post')

    character_level_input_sequences.append(character_level_input_sequence)

    input_words_lengths.append(words_lengths)

# Get word-level sentences lengths.
input_sentences_lengths = []
for sequence in input_sequences: input_sentences_lengths.append(
    (len(sequence) if len(sequence) <= config.MAX_SENTENCE_LENGTH
     else config.MAX_SENTENCE_LENGTH))

# Add padding for each.
input_sequences = pad_sequences(input_sequences, maxlen=config.MAX_SENTENCE_LENGTH, padding='post', truncating='post')
output_sequences = pad_sequences(output_sequences, maxlen=config.MAX_SENTENCE_LENGTH, padding='post', truncating='post')
character_level_input_sequences = pad_sequences(character_level_input_sequences, maxlen=config.MAX_SENTENCE_LENGTH,
                                                padding='post', truncating='post')
input_words_lengths = pad_sequences(input_words_lengths, maxlen=config.MAX_SENTENCE_LENGTH, padding='post',
                                    truncating='post')

input_sequences_np = np.array(input_sequences)
character_level_input_sequences_np = np.array(character_level_input_sequences)
output_sequences_np = np.array(output_sequences)

input_words_lengths_np = np.array(input_words_lengths)
input_sentences_lengths_np = np.array(input_sentences_lengths)

In [6]:
(input_sentences_lengths[0])

32

In [7]:
# def prepare_batch(input, output):
#     word_level_input = word_level_tokenizer.texts_to_sequences(input)
#     word_level_input = tf.ragged.constant(word_level_input)
#     word_level_input = word_level_input[:, :MAX_WORD_LEVEL_TOKENS]
#     word_level_input = word_level_input.to_tensor()
# 
#     unk_input_sequences = word_unk_level_tokenizer.texts_to_sequences(input)
#     character_level_input_sequences = []
# 
#     for sequence in unk_input_sequences:
#         character_level_input_sequence = []
#         for word_token in sequence:
#             word = word_unk_level_tokenizer.index_word[word_token]
#             word = character_level_tokenizer.texts_to_sequences(word)
#             word_chars = [each[0] for each in word]
#             character_level_input_sequence.append(word_chars)
#         character_level_input_sequence = tf.ragged.constant(character_level_input_sequence)
#         character_level_input_sequence = character_level_input_sequence[
#                                          :MAX_SENTENCE_LENGTH,
#                                          :MAX_WORD_LENGTH]
#         character_level_input_sequence = character_level_input_sequence.to_tensor()
#         character_level_input_sequences.append(character_level_input_sequence)
# 
#     print(character_level_input_sequences)
#     character_level_input_sequences = tf.ragged.constant(character_level_input_sequences)
# 
#     output = word_level_tokenizer.texts_to_sequences(output)
#     output = tf.ragged.constant(output)
#     output = output[:, :MAX_SENTENCE_LENGTH]
#     output = output.to_tensor()
# 
#     return (word_level_input, character_level_input_sequences), output

In [8]:
from HierarchicalTransformerEncoder import HierarchicalTransformerEncoder

model = HierarchicalTransformerEncoder(num_character_level_layers=config.NUM_CHARACTER_LEVEL_LAYERS,
                                       num_word_level_layers=config.NUM_WORD_LEVEL_LAYERS,
                                       character_level_d_model=config.CHARACTER_LEVEL_D_MODEL,
                                       word_level_d_model=config.WORD_LEVEL_D_MODEL,
                                       num_heads=config.NUM_HEADS, dff=config.DFF,
                                       max_word_length=config.MAX_WORD_LENGTH,
                                       max_sentence_length=config.MAX_SENTENCE_LENGTH,
                                       vocab_size=config.VOCAB_SIZE,
                                       character_vocab_size=config.CHARACTER_VOCAB_SIZE)

In [9]:
from CustomSchedule import CustomSchedule

learning_rate = CustomSchedule(config.WORD_LEVEL_D_MODEL)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [10]:

# word_input_shape = (config.BATCH_SIZE, config.MAX_SENTENCE_LENGTH)
# char_input_shape = (config.BATCH_SIZE, config.MAX_SENTENCE_LENGTH, config.MAX_WORD_LENGTH)
# 
# model.build(input_shape=[word_input_shape, char_input_shape])

def custom_loss(y_true, y_pred):
    print('Find loss:')
    print("Shape y_true:", y_true[0].shape)
    print("Shape y_pred:", y_pred[0].shape)
    softmax_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)(y_true[0], y_pred[0])
    print('loss = ', softmax_loss)
    # sigmoid_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)(y_true, y_pred[1])
    total_loss = softmax_loss
    return total_loss


model.compile(optimizer=optimizer, loss=custom_loss, metrics=['acc'])
# model.summary()

In [11]:
print(input_sequences_np.shape)
print(input_sentences_lengths_np.shape)
print(character_level_input_sequences_np.shape)
print(input_words_lengths_np.shape)

(100, 40)
(100,)
(100, 40, 10)
(100, 40)


In [12]:
model.fit(
    [[input_sequences_np, input_sentences_lengths_np], [character_level_input_sequences_np, input_words_lengths_np]],
    output_sequences_np, epochs=config.EPOCHS,
    batch_size=config.BATCH_SIZE)

Epoch 1/10
Shape của word_embedding_outputs: (50, 40, 128)
Instructions for updating:
Use fn_output_signature instead
Shape của correction_output: (50, 40, 10000)
Shape của detection_output: (50, 40)
Find loss:
Shape y_true: (40,)
Shape y_pred: (40, 10000)
loss =  Tensor("custom_loss/sparse_categorical_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Shape của word_embedding_outputs: (50, 40, 128)
Shape của correction_output: (50, 40, 10000)
Shape của detection_output: (50, 40)
Find loss:
Shape y_true: (40,)
Shape y_pred: (40, 10000)
loss =  Tensor("custom_loss/sparse_categorical_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
2/2 [==============================] - 7s 456ms/step - loss: 9.1723 - acc: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 1s 415ms/step - loss: 9.1394 - acc: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 1s 423ms/step - loss: 9.2358 - acc: 3.9888e-04
Epoch 4/10
2/2 [==============================] - 1s 436ms

# Test model

In [13]:
test_output = model.predict([[input_sequences_np[:100], input_sentences_lengths_np[:100]],
                             [character_level_input_sequences_np[:100], input_words_lengths_np[:100]]])

Shape của word_embedding_outputs: (None, 40, 128)
Shape của correction_output: (None, 40, 10000)
Shape của detection_output: (None, 40)
4/4 [==============================] - 1s 56ms/step


In [14]:
for sentence in test_output[:100]:
    out = ''
    for word in sentence:
        index = tf.argmax(word, axis=0).numpy()
        word_str = word_level_tokenizer.index_word.get(index)
        if word_str is not None:
            out += word_str + ' '
        else:
            out += '<UNK> '
    print(out)

<UNK> <UNK> <UNK> <UNK> <UNK> do <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 
<UNK> <UNK> <UNK> <UNK> <UNK> do <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 
<UNK> <UNK> <UNK> <UNK> <UNK> do <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 
<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 
<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UN

In [15]:
model.save('model.tf')

Shape của word_embedding_outputs: (None, 40, 128)
Shape của correction_output: (None, 40, 10000)
Shape của detection_output: (None, 40)
Shape của word_embedding_outputs: (None, 40, 128)
Shape của correction_output: (None, 40, 10000)
Shape của detection_output: (None, 40)
Shape của word_embedding_outputs: (None, 40, 128)
Shape của correction_output: (None, 40, 10000)
Shape của detection_output: (None, 40)
INFO:tensorflow:Assets written to: model.tf/assets


INFO:tensorflow:Assets written to: model.tf/assets


In [16]:
loaded_model = tf.keras.models.load_model('model.tf', custom_objects={'CustomSchedule': CustomSchedule})

ValueError: Unknown loss function: 'custom_loss'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
loaded_model.predict([[input_sequences_np[:100], input_sentences_lengths_np[:100]],
                      [character_level_input_sequences_np[:100], input_words_lengths_np[:100]]])